

---

## Проект 1

### Сопоставительный анализ простых и сложных ансамблей архитектур нейронных сетей для решения задачи регрессии.

### Этап 5. Обучение анасамблей сложных моделей.

### Цели и задачи этапа

Обучение ансамблей простых моделей полученных на этапе 2.

### IMPORT LIBRARIES

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Concatenate, Average
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

## IMPORT DATA
---

In [35]:
data_path = r'/Users/rttrif/Data_Science_Projects/Internship_MindSet/Project_1_Regression_task'

In [36]:
# Read X_train
X_train = pd.read_csv(data_path + '/X_train.csv')
X_train.head()

,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,0.084200,236.878667,10.953333,0.0,36.966667,0.600000
1,0.232600,239.185000,4.173333,0.0,0.000000,8.433333
2,0.138600,244.615333,1.420000,0.0,0.500000,0.000000
3,0.015800,236.114667,11.366667,8.5,0.000000,17.666667
4,0.079133,240.075667,6.253333,0.0,0.000000,20.433333


In [37]:
# Read X_test
X_test = pd.read_csv(data_path + '/X_test.csv')
X_test.head()

,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,0.040733,238.253333,11.300000,0.0,0.866667,17.433333
1,0.064333,233.142333,7.660000,0.0,0.333333,16.666667
2,0.058000,242.286667,1.213333,0.0,0.000000,0.633333
3,0.099667,243.178667,1.506667,0.0,0.133333,0.000000
4,0.117733,233.769333,6.026667,0.0,0.366667,0.000000


In [38]:
# Read y_train
y_train = pd.read_csv(data_path + '/y_train.csv')
y_train.head()

,Global_active_power
0,2.572000
1,0.951533
2,0.336667
3,2.684667
4,1.501467


In [39]:
# Read y_test
y_test =  pd.read_csv(data_path + '/y_test.csv')
y_test.head()

,Global_active_power
0,2.693333
1,1.794067
2,0.265600
3,0.346267
4,1.407400


### Normalize data

In [40]:
# scaler
scaler = MinMaxScaler(feature_range=(0, 1))

In [41]:
# Normalize X_train
X_train_norm = scaler.fit_transform(X_train)
print(X_train.shape)

(47840, 6)


In [42]:
X_train_norm

array([[0.10669933, 0.4204013 , 0.333611  , 0.        , 0.65427729,
        0.02524544],
       [0.29475374, 0.50663011, 0.12182424, 0.        , 0.        ,
        0.35483871],
       [0.17563572, 0.70965852, 0.03581841, 0.        , 0.00884956,
        0.        ],
       ...,
       [0.06023486, 0.71335992, 0.03061224, 0.        , 0.        ,
        0.02805049],
       [0.16684971, 0.76855683, 0.03456893, 0.        , 0.        ,
        0.        ],
       [0.45146574, 0.31360917, 0.4606414 , 0.26985797, 0.        ,
        0.        ]])

In [43]:
# Normalize X_test
X_test_norm = scaler.fit_transform(X_test)
print(X_test.shape)

(20484, 6)


In [44]:
X_test_norm

array([[0.05737089, 0.46502294, 0.32828987, 0.        , 0.01424658,
        0.74395448],
       [0.09061033, 0.26520799, 0.21958989, 0.        , 0.00547945,
        0.71123755],
       [0.08169014, 0.62270642, 0.02707545, 0.        , 0.        ,
        0.02702703],
       ...,
       [0.08873239, 0.5500417 , 0.05853076, 0.        , 0.        ,
        0.07965861],
       [0.20065728, 0.46528357, 0.49253434, 0.        , 0.01150685,
        0.74110953],
       [0.13915493, 0.49091691, 0.49930321, 0.        , 0.47780822,
        0.18918919]])

## Training ensembles of simple models

Обучение ансамля ансамблей. Беруться финальные модели каждой из групп т.е. анасамбли всех простых моделей.

### Model_13a

Анасамбль моледей: 6a + 6b

In [45]:
# Input for model_6a
input = Input(shape = (6,))

# Input for model_6b
input_A = Input(shape = (6,),name="wide_input")
input_B = Input(shape = (6,),name="deep_input")

# Load models
model_6a = tf.keras.models.load_model('model_6a.h5')(input)
model_6b = tf.keras.models.load_model('model_6b.h5')(input_A, input_B)

# Output
output = Average()([model_6a, model_6b])
output = Dense(1, activation='linear', name="output")(output)

model_13a = Model(inputs= [input], outputs=[output])

# Compile model
model_13a.compile(optimizer='Adam',
                 loss='mse',
                 metrics=['mae'])


model_13a.summary()

tf.keras.utils.plot_model(model_13a)

AssertionError: Could not compute output KerasTensor(type_spec=TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), name='output/BiasAdd:0', description="created by layer 'output'")

In [ ]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("model_12.h5",
                                                   save_best_only=True)

early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10,
                                                     restore_best_weights=True)

history_12 = model_12.fit(X_train_norm, y_train,
                          epochs=200,
                          batch_size=128,
                          verbose=2,
                          callbacks =[checkpoint_cb, early_stopping_cb],
                          validation_split=0.2)

#### Оценка результатов

In [ ]:
# Learning curves
pd.DataFrame(history_12.history).plot(figsize=(20, 8))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
fig, (axL, axR) = plt.subplots(ncols=2, figsize=(20, 8))
axL.plot(history_12.history['loss'], label="loss (mse) for training")
axL.plot(history_12.history['val_loss'], label="loss (mse) for validation")
axL.set_title('model loss')
axL.set_xlabel('epoch')
axL.set_ylabel('loss')
axL.legend(loc='upper right')

axR.plot(history_12.history['mae'], label="mae for training")
axR.plot(history_12.history['val_mae'], label="mae for validation")
axR.set_title('model mae')
axR.set_xlabel('epoch')
axR.set_ylabel('mae')
axR.legend(loc='upper right')

plt.show()

In [ ]:
# Evaluate the model on the test set
evaluate_12 = model_12.evaluate(X_test, y_test, verbose=2)

In [ ]:
# Predictions
y_preds_train = model_12.predict(X_train)
y_preds_test = model_12.predict(X_test)

# calculate root mean squared error
trainScore_12 = math.sqrt(mean_squared_error(y_train, y_preds_train))
print('Train Score: %.2f RMSE' % (trainScore_12))

testScore_12 = math.sqrt(mean_squared_error(y_test, y_preds_test))
print('Test Score: %.2f RMSE' % (testScore_12))

---